In [19]:
import requests
from bs4 import BeautifulSoup

headers = {"User-Agent": "Mozilla/5.0"}

def get_product_links_from_page(url):
    res = requests.get(url, headers=headers)
    soup = BeautifulSoup(res.text, "html.parser")
    product_links = [a["href"].strip() for a in soup.select("a.product-item-link") if a.get("href")]
    return product_links

def scrape_product_details(product_url):
    res = requests.get(product_url, headers=headers)
    soup = BeautifulSoup(res.text, "html.parser")

    def get_text(selector):
        tag = soup.select_one(selector)
        return tag.get_text(strip=True) if tag else "N/A"

    title = get_text("h1.page-title span.base")
    overview = get_text("div.product.attribute.overview div.value")
    price = get_text("span.price")
    stock_tag = soup.select_one("div.product-info-stock-sku div.stock")
    stock = stock_tag.get_text(strip=True) if stock_tag else "N/A"
    description = get_text("div.product.attribute.description div.value")

    product_data = {
        "title": title,
        "overview": overview,
        "price": price,
        "stock": stock,
        "description": description,
        "link": product_url,
    }

    # More Information table
    rows = soup.select("table#product-attribute-specs-table tr")
    for row in rows:
        try:
            key = row.select_one("th").text.strip()
            value = row.select_one("td").text.strip()
            product_data[key] = value
        except:
            continue

    return product_data

# === MAIN SCRAPER ===
base_url = "https://www.freshpoolsupply.com/chemical/chlorine.html"
page = 1
all_product_links = []

while True:
    paged_url = f"{base_url}?p={page}"
    print(f"Scraping page {page} → {paged_url}")
    links = get_product_links_from_page(paged_url)
    if not links:
        print("No more products found. Ending pagination loop.")
        break
    all_product_links.extend(links)
    page += 1

print(f"\nTotal product links collected: {len(all_product_links)}")

# Scrape each product
all_products = []
for i, link in enumerate(all_product_links, 1):
    print(f"Scraping product {i}/{len(all_product_links)}: {link}")
    product_data = scrape_product_details(link)
    all_products.append(product_data)

# Preview first product (optional)
print("\n📦 Sample Product:\n", all_products[0])


Scraping page 1 → https://www.freshpoolsupply.com/chemical/chlorine.html?p=1
Scraping page 2 → https://www.freshpoolsupply.com/chemical/chlorine.html?p=2
Scraping page 3 → https://www.freshpoolsupply.com/chemical/chlorine.html?p=3
Scraping page 4 → https://www.freshpoolsupply.com/chemical/chlorine.html?p=4
Scraping page 5 → https://www.freshpoolsupply.com/chemical/chlorine.html?p=5
Scraping page 6 → https://www.freshpoolsupply.com/chemical/chlorine.html?p=6
Scraping page 7 → https://www.freshpoolsupply.com/chemical/chlorine.html?p=7
Scraping page 8 → https://www.freshpoolsupply.com/chemical/chlorine.html?p=8
No more products found. Ending pagination loop.

Total product links collected: 73
Scraping product 1/73: https://www.freshpoolsupply.com/chemical/chlorine/ab-3-tabs-50-lbalr.html
Scraping product 2/73: https://www.freshpoolsupply.com/chemical/chlorine/ab-granular-dichlor-50-lbloq.html
Scraping product 3/73: https://www.freshpoolsupply.com/chemical/chlorine/algaecure-25-lbrrm.html


In [20]:
all_products

[{'title': 'AB 3" Tabs 50 lb | 40631A',
  'overview': 'Applied Biochemists 3" Tablets',
  'price': '$284.95',
  'stock': 'Out of stock',
  'description': 'Applied Biochemists® 3” TabletsSlow dissolving, 3” stabilized chlorinating tablets contain 90% available chlorine to effectively sanitize and maintain consistent chlorine levels.When using 3” Tablets, shock treat at least once a week using Applied Biochemists® Power Blast 70 GranularFor best results, add tablets to a new, clean skimmer, floater or automatic feederTest the chlorine and pH levels at least twice a week and adjust as needed',
  'link': 'https://www.freshpoolsupply.com/chemical/chlorine/ab-3-tabs-50-lbalr.html',
  'Manufacturer SKU': '40631A',
  'Size': '50 lb',
  'Brand': 'APPLIED BIO'},
 {'title': 'AB Granular Dichlor 50 lb | 40697A',
  'overview': 'Applied Biochemists Granular Chlorine',
  'price': '$356.95',
  'stock': 'Out of stock',
  'description': 'Applied Biochemists® Granular ChlorineThis stabilized and concentr